# [ 가위바위보 분류기 ]

## Step_0. 필요한 패키지 import

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import os, glob

print("PIL library import complete")

PIL library import complete


## Step_1. train data 준비

In [2]:
# x_train 이미지 28x28 사이즈로 변경

import os
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME")+"/practice/rock_scissor_paper"
print("=> 이미지 디렉토리 경로:", image_dir_path)

images_scissor = glob.glob(image_dir_path + "/train_set/scissor_jpg/*.*")
images_rock = glob.glob(image_dir_path + "/train_set/rock_jpg/*.*")
images_paper = glob.glob(image_dir_path + "/train_set/paper_jpg/*.*")

# 파일마다 모두 28X28 사이즈로 바꾸어 저장
target_size = (28, 28)

for img in images_scissor:
    old_img = Image.open(img)                                 # 가위 이미지를 old_img에 저장
    new_img = old_img.resize(target_size, Image.ANTIALIAS)    # 사이즈를 재조정 해서 new_img에 저장
    new_img.save(img,"JPEG")                                  # jpg와 jpeg는 같은 파일이지만 변경된 이미지를 JPEG 확장자로 통일
print("가위 이미지 resize 완료!")

for img in images_rock:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("바위 이미지 resize 완료!")

for img in images_paper:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("보 이미지 resize 완료!")

=> 이미지 디렉토리 경로: /home/myungjin-kim/practice/rock_scissor_paper
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


In [3]:
# x_train, y_train 지정 
# y_train 라벨링

def load_data(img_path):
    
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data = 7000  # 총 이미지 개수
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/practice/rock_scissor_paper/train_set"
(x_train, y_train) = load_data(image_dir_path)

x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 6221 입니다.
x_train shape: (7000, 28, 28, 3)
y_train shape: (7000,)


In [4]:
print('x_train의 최솟값:', np.min(x_train), ', x_train의 최댓값:', np.max(x_train))

print('x_train_norm의 최솟값:', np.min(x_train_norm), ', x_train_norm의 최댓값:', np.max(x_train_norm))

x_train의 최솟값: 0 , x_train의 최댓값: 255
x_train_norm의 최솟값: 0.0 , x_train_norm의 최댓값: 1.0


## Step_2. model 설계

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model의 입력/출력부(가위바위보 데이터셋 Vs MNIST 데이터셋) 유의하기.
# hyper parameter tunning
n_channel_1 = 32
n_channel_2 = 64
n_channel_3 = 64
n_dense = 64
n_train_epoch = 20

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='sigmoid'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

## Step_3. model training

In [6]:
# model.compile(), model.fit()

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs = n_train_epoch)


Epoch 1/20
219/219 [==============================] - 3s 13ms/step - loss: 1.0009 - accuracy: 0.4234
Epoch 2/20
219/219 [==============================] - 3s 11ms/step - loss: 0.9019 - accuracy: 0.5254
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.6857 - accuracy: 0.6886
Epoch 4/20
219/219 [==============================] - 3s 12ms/step - loss: 0.5024 - accuracy: 0.7981
Epoch 5/20
219/219 [==============================] - 3s 12ms/step - loss: 0.3804 - accuracy: 0.8530
Epoch 6/20
219/219 [==============================] - 3s 12ms/step - loss: 0.3184 - accuracy: 0.8759
Epoch 7/20
219/219 [==============================] - 3s 11ms/step - loss: 0.2553 - accuracy: 0.9010
Epoch 8/20
219/219 [==============================] - 3s 11ms/step - loss: 0.2143 - accuracy: 0.9160
Epoch 9/20
219/219 [==============================] - 3s 12ms/step - loss: 0.1917 - accuracy: 0.9257
Epoch 10/20
219/219 [==============================] - 3s 12ms/step - loss: 0.1625 - accura

## Step_4. test data 준비

In [7]:
# x_test 이미지도 28x28 사이즈로 변경

import os
image_dir_path = os.getenv("HOME")+"/practice/rock_scissor_paper"
print("=> 이미지 디렉토리 경로:", image_dir_path)

images_scissor = glob.glob(image_dir_path + "/test_set/scissor/*.jpg")
images_rock = glob.glob(image_dir_path + "/test_set/rock/*.jpg")
images_paper = glob.glob(image_dir_path + "/test_set/paper/*.jpg")


target_size = (28, 28)

for img in images_scissor:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("가위 이미지 resize 완료!")

for img in images_rock:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("바위 이미지 resize 완료!")

for img in images_paper:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("보 이미지 resize 완료!")


=> 이미지 디렉토리 경로: /home/myungjin-kim/practice/rock_scissor_paper
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


In [8]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data = 611  # 가위바위보 이미지 개수 총합
    img_size = 28
    color = 3
    #이미지 데이터와 라벨(가위: 0, 바위: 1, 보: 2) 데이터를 담을 행렬 생성
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img   
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img   
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/practice/rock_scissor_paper/test_set"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/208.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 611 입니다.
x_test shape: (611, 28, 28, 3)
y_test shape: (611,)


In [9]:
print('x_test의 최솟값:', np.min(x_test), ', x_test의 최댓값:', np.max(x_test))

print('x_test_norm의 최솟값:', np.min(x_test_norm), ', x_test_norm의 최댓값:', np.max(x_test_norm))

x_test의 최솟값: 0 , x_test의 최댓값: 255
x_test_norm의 최솟값: 0.0 , x_test_norm의 최댓값: 1.2259615384615385


## Step_5. test

In [10]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print()

20/20 - 0s - loss: 2.0182 - accuracy: 0.7267



## [ 진행사항 ]

1. 1차시도

train data = 3080 (정규화 완료) / test data = 311 (정규화 완료)

n_channel_1 = 32
n_channel_2 = 64
n_dense = 32
n_train_epoch = 15

=> result: loss: 5.9083 / accuracy: 0.6592


2. 2차시도

train data = 5020 (정규화 완료) / test data = 451 (정규화 완료)

n_channel_1 = 32
n_channel_2 = 64
n_dense = 32
n_train_epoch = 17

=> result: loss: 5.9083 / accuracy: 0.6592

2. 3차시도

train data = 6221 (정규화 완료) / test data = 611 (정규화 완료)

n_channel_1 = 32
n_channel_2 = 64
n_channel_3 = 64
n_dense = 64
n_train_epoch = 20

model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='sigmoid')) # activation 함수 변경
model.add(keras.layers.MaxPooling2D((2,2)))                              # 추가
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))    # 추가

=> result: loss: 2.0182 / accuracy: 0.7267

## [ 회 고 ]

1. 이번 프로젝트에서 어려웠던 점: parameter 조정이 어려움


2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.


- 알아낸 점: png파일을 jpg파일로 바꾸는 방법, 분류 시스템 전반적 구축 방법, 정규화, hyperparameter조정


- 모호한 점: train 1차 학습 후 test 1차 시도시 6.0이 넘는 accuracy가 나왔는데 train data와 test data를 늘리고 epoch와 conv2D를 증가시켜 모델을 돌렸더니 오히려 train 2차 시도시 accaracy가 1.0이 나오고 test 2차 결과는 0.5 정로 떨어졌다. 그러자 3차 시도때는 자료를 처음의 거의 두배로 늘려주고 아래 시도와 같이 하니 다시 accaracy가 7.0이 넘게 나왔다. 그런데 2차 시도시 도대체 왜 accuracy가 떨어졌는지.. 그리고 이럴때는 과적합이 맞는건지 또한 과적합을 이 모델에서 해결할 수 있는 방법은 오로지 데이터의 증가일 뿐인건지가 모호함.


3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.
  - train_data_set과 test_data_set의 개수를 2배 가량으로 늘려줌.
  - model의 depth를 늘려줌
  - activation function 변경
  - epoch 늘려줌